In [1]:
import pandas as pd
import requests

def geocode_city_state(city_state):
    url = f"https://nominatim.openstreetmap.org/search?format=json&q={city_state}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data:
            return data[0]['lat'], data[0]['lon']
    return None, None

original_csv_file = 'cleaned_cham_api_1900_1963_dates.csv'
data = pd.read_csv(original_csv_file)

def extract_coordinates(row):
    place = row['place']
    longitude, latitude = geocode_city_state(place)
    return longitude, latitude

data['reclong'], data['reclat'] = zip(*data.apply(extract_coordinates, axis=1))

new_csv_file = 'cham_geo_dataset_YYYY.csv'
data.to_csv(new_csv_file, index=False)

print("new csv success")


/var/folders/lq/7w27s5ld7fl2p8px3kczqn6m0000gn/T/ipykernel_40163/2026514625.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


New CSV file has been created with latitude and longitude columns.


In [3]:
import pandas as pd
import json

def csv_to_geojson(csv_file):
    data = pd.read_csv(csv_file)
    
    features = []
    for _, row in data.iterrows():
        # Check if location coordinates are not NaN
        if not pd.isna(row['reclong']) and not pd.isna(row['reclat']):
            feature = {
                'type': 'Feature',
                'properties': {
                    'date': row['date'],
                    'title': row['title'],
                    'id': row['id'],
                    'place': row['place'],
                },
                'geometry': {
                    'type': 'Point',
                    'coordinates': [row['reclong'], row['reclat']]
                }
            }
            features.append(feature)
    
    geojson_data = {
        'type': 'FeatureCollection',
        'features': features
    }
    
    return json.dumps(geojson_data)

csv_file = 'cham_geo_dataset_YYYY.csv'
geojson = csv_to_geojson(csv_file)

output_geojson_file = 'cham_1900_1963_YYYY.geojson'
with open(output_geojson_file, 'w') as f:
    f.write(geojson)

print("success")


success
